In [3]:
import yfinance as yf
import pandas as pd
import time
import tqdm
import json
pd.options.display.float_format = "{:,.0f}".format

In [4]:
url = "https://raw.githubusercontent.com/rreichel3/US-Stock-Symbols/main/nyse/nyse_full_tickers.json"
df_nyse = pd.read_json(url)
url_nasdaq = "https://raw.githubusercontent.com/rreichel3/US-Stock-Symbols/refs/heads/main/nasdaq/nasdaq_full_tickers.json"
df_nasdaq = pd.read_json(url_nasdaq)

df_raw = pd.concat([df_nyse, df_nasdaq], ignore_index=True)

# Remove symbols with '^' character
df_raw = df_raw[df_raw["symbol"].str.contains(r"\^") == False]

# Ensure market cap is numeric. Drop NA
df_raw["marketCap"] = pd.to_numeric(df_raw["marketCap"], errors='coerce')
df_raw["symbol"] = df_raw["symbol"].str.replace("/", "-", regex=False)
df_raw.dropna(subset=["marketCap"], inplace=True)
df_raw = df_raw[df_raw.marketCap > 10000000]

accepted_countries = set(['United States', 'Canada', 'United Kingdom', 'Germany', 'France', 'Italy', 'Spain', 'Netherlands', 'Switzerland', 'Australia', 'Japan', 'Kazakhstan', 'Ireland', 'Sweden', 'Denmark', 'Norway', 'Finland'])
df_raw = df_raw[df_raw.country.isin(accepted_countries)]

df = df_raw[['symbol', 'name', 'marketCap', 'country', 'ipoyear', 'industry', 'sector']].copy()
df.head(3)

,symbol,name,marketCap,country,ipoyear,industry,sector
0,A,Agilent Technologies Inc. Common Stock,"41,903,968,123",United States,1999,Biotechnology: Laboratory Analytical Instruments,Industrials
1,AA,Alcoa Corporation Common Stock,"16,247,403,368",United States,2016,Aluminum,Industrials
3,AAMI,Acadian Asset Management Inc. Common Stock,"1,730,821,046",United Kingdom,2014,Investment Managers,Finance


In [5]:
df[df.symbol.str.contains(r"BRK")]

,symbol,name,marketCap,country,ipoyear,industry,sector
419,BRK-A,Berkshire Hathaway Inc.,"1,095,801,130,000",United States,,,
420,BRK-B,Berkshire Hathaway Inc.,"1,094,595,197,381",United States,,,
2048,RBRK,Rubrik Inc. Class A Common Stock,"15,384,880,564",United States,2024,Computer Software: Prepackaged Software,Technology
3308,BRKR,Bruker Corporation Common Stock,"8,054,400,043",United States,,Biotechnology: Laboratory Analytical Instruments,Industrials
3309,BRKRP,Bruker Corporation 6.375% Mandatory Convertibl...,"59,650,573,723",United States,,Biotechnology: Laboratory Analytical Instruments,Industrials


In [6]:
dats = yf.Tickers(df.symbol.to_list())

In [ ]:
all_info = []
# with open("company_info.json", "w") as f:
for symbol in tqdm.tqdm(df.symbol.to_list()):
    try:
        info = dats.tickers[symbol].info
        all_info.append(info)
        time.sleep(0.2)  # To avoid hitting rate limits
    except Exception as e:
        # f.write(f"{symbol}: ERROR - {str(e)}\n")
        # time.sleep(0.2)  # To avoid hitting rate limits
        print(symbol, "ERROR:", str(e))

json.dump(all_info, open("company_info.json", "w"), indent=4)

  5%|▍         | 203/4397 [01:41<34:36,  2.02it/s] 